In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import os

# Parameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10

TRAIN_DIR = "D:/SmartTreeSurvey/Dataset/Train"
VAL_DIR = "D:/SmartTreeSurvey/Dataset/test"

# Image augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    zoom_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Load data
train_data = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'  # changed from 'binary'
)

val_data = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'  # changed from 'binary'
)

# Load base MobileNetV2 model
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(4, activation='softmax')(x)  # changed from 1->4 and sigmoid->softmax

model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(train_data, validation_data=val_data, epochs=EPOCHS)

# Save model
model.save("soil_model.h5")
print("✅ Model training complete! Saved as 'soil_model.h5'")


Found 1214 images belonging to 4 classes.
Found 339 images belonging to 4 classes.
Epoch 1/10


c:\Users\aab\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


38/38 ━━━━━━━━━━━━━━━━━━━━ 66s 2s/step - accuracy: 0.6386 - loss: 0.9230 - val_accuracy: 0.8024 - val_loss: 0.5406
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - accuracy: 0.8642 - loss: 0.3792 - val_accuracy: 0.8732 - val_loss: 0.3893
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.9206 - loss: 0.2487 - val_accuracy: 0.8525 - val_loss: 0.4165
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.9402 - loss: 0.1785 - val_accuracy: 0.8230 - val_loss: 0.5317
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - accuracy: 0.9399 - loss: 0.1671 - val_accuracy: 0.8938 - val_loss: 0.2663
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.9549 - loss: 0.1356 - val_accuracy: 0.9145 - val_loss: 0.2442
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.9634 - loss: 0.1200 - val_accuracy: 0.8968 - val_loss: 0.2856
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.9727 - loss: 0.1041 - val_accuracy: 0.9233 - val_loss: 0.1819
Epo

✅ Model training complete! Saved as 'soil_model.h5'


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the model
model = tf.keras.models.load_model("soil_model.h5")

# Load and preprocess a single image
img_path = "D:\SmartTreeSurvey\plant_photo.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Make prediction
prediction = model.predict(img_array)

# Assuming 4 classes, update these names to match your actual folder names
class_names = class_names = ['alluvial', 'black', 'clay', 'red']

# Get index of highest probability
predicted_index = np.argmax(prediction[0])
predicted_class = class_names[predicted_index]

print("Predicted class:", predicted_class)


<>:9: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\S'
C:\Users\aab\AppData\Local\Temp\ipykernel_28664\770336629.py:9: SyntaxWarning: invalid escape sequence '\S'
  img_path = "D:\SmartTreeSurvey\plant_photo.jpg"


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted class: red
